In [3]:
import numpy as np
import qibo
from qibo.symbols import X, Z, I
from qibo import models, gates, hamiltonians

import cma
from scipy.optimize import minimize
from shapely.geometry import Polygon, Point

qibo.set_backend("numpy")

[Qibo 0.1.11|INFO|2023-12-20 22:26:20]: Using numpy backend on /CPU:0


In [2]:
num_data = 15   # How many ground states do we want for training?
nqubits = 2     # num qubits
accuracy_training = 100 # Maximum acc percentage

layers = int(np.log2(nqubits))
nparams = 30*(layers-1) + 15

# Gound states

In [3]:
def ground_state(j1, j2):
    
    symbolic_expr = 0
    for i in range(nqubits):
        symbolic_expr += Z(i)
        symbolic_expr -= j1*X(i)*X((i+1)%nqubits)
        symbolic_expr -= j2*X(i)*Z((i+1)%nqubits)*X((i+2)%nqubits)
    
    hamiltonian = hamiltonians.SymbolicHamiltonian(form=symbolic_expr)
    ground_state = hamiltonian.ground_state()
    
    return ground_state

In [4]:
def labeling(x, y):
    # Definir las coordenadas de los puntos de cada región
    region1_coords = [(-2, 1), (2, 1), (0, -1)]
    region2_coords = [(0, -1), (3, -4), (4, -4), (4, 3)]
    region3_coords = [(0, -1), (-3, -4), (-4, -4), (-4, 3)]
    region4_coords = [(-2, 1), (2, 1), (4, 3), (4, 4), (-4, 4), (-4, 3)]
    region5_coords = [(-3, -4), (0, -1), (3, -4)]
    
    # Crear objetos Polygon para cada región
    region1_poly = Polygon(region1_coords)
    region2_poly = Polygon(region2_coords)
    region3_poly = Polygon(region3_coords)
    region4_poly = Polygon(region4_coords)
    region5_poly = Polygon(region5_coords)
    
    punto = Point(x, y)
    if region1_poly.contains(punto):
        return 3
    elif region2_poly.contains(punto):
        return 1
    elif region3_poly.contains(punto):
        return 2
    elif region4_poly.contains(punto):
        return 0
    elif region5_poly.contains(punto):
        return 0
    else:
        return None # Si el punto no está en ninguna región

In [5]:
gs_list = []
label_list = []
j1_list = np.random.uniform(-4, 4, num_data)
j2_list = np.random.uniform(-4, 4, num_data)

for i in range(num_data):
    gs_list.append(ground_state(j1_list[i], j2_list[i]))
    label_list.append(labeling(j1_list[i], j2_list[i]))

[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.3|WARNING|2023-12-20 09:57:53]: Calculating the dens

In [15]:
gs_list = np.array(gs_list)
print(gs_list.shape)

(15, 4)


# CNN

In [7]:
def convolutional_layer(c, q1, q2, param):
    c.add(gates.U3(q1, theta=param[0], phi=param[1], lam=param[2]))
    c.add(gates.U3(q1, theta=param[3], phi=param[4], lam=param[5]))
    c.add(gates.CNOT(q2, q1))
    c.add(gates.RZ(q1, theta=param[6]))
    c.add(gates.RY(q2, theta=param[7]))
    c.add(gates.CNOT(q1, q2))
    c.add(gates.RY(q2, theta=param[8]))
    c.add(gates.CNOT(q2, q1))
    c.add(gates.U3(q1, theta=param[9], phi=param[10], lam=param[11]))
    c.add(gates.U3(q1, theta=param[12], phi=param[13], lam=param[14]))
    return c

def pooling_layer(c, q1, q2, param):
    c.add(gates.U3(q1, theta=param[0], phi=param[1], lam=param[2]))
    c.add(gates.U3(q1, theta=param[3], phi=param[4], lam=param[5]))
    c.add(gates.CNOT(q2, q1))
    c.add(gates.RZ(q1, theta=param[6]))
    c.add(gates.RY(q2, theta=param[7]))
    c.add(gates.CNOT(q1, q2))
    c.add(gates.RY(q2, theta=param[8]))
    c.add(gates.CNOT(q2, q1))
    c.add(gates.U3(q1, theta=param[9], phi=param[10], lam=param[11]))
    c.add(gates.U3(q1, theta=param[12], phi=param[13], lam=param[14]))
    return c

In [8]:
def cnn(params):
    
    circuit = models.Circuit(nqubits)
    qubits = list(range(nqubits))
    
    for j in range(layers-1):
        
        len_qubits = len(qubits)
        
        for i in range(len_qubits//2):
            convolutional_layer(circuit, qubits[2*i], qubits[(2*i+1)%len_qubits], params[15*2*j:15*(2*j+1)])
        
        for i in range(len_qubits//2):
            convolutional_layer(circuit, qubits[2*i+1], qubits[(2*i+2)%len_qubits], params[15*2*j:15*(2*j+1)])
            
        for i in range(len_qubits//2):
            pooling_layer(circuit, qubits[2*i], qubits[(2*i+1)%len_qubits], params[15*(2*j+1):15*(2*j+2)])

        qub = []
        for i in range(len_qubits):
            if i%2 == 1:
                qub.append(qubits[i])
                
        qubits = qub
    
    convolutional_layer(circuit, qubits[0], qubits[1], params[15*(2*layers-2):15*(2*layers-1)])

    return circuit

In [9]:
params = np.random.uniform(0, 2*np.pi, nparams)
c = cnn(params)
print(c.draw())

q0: ─U3─U3─X─RZ─o────X─U3─U3─
q1: ───────o─RY─X─RY─o───────


# Loss and accuracy

In [10]:
ham7 = hamiltonians.SymbolicHamiltonian(form=Z(nqubits-1))
ham3 = hamiltonians.SymbolicHamiltonian(form=Z(nqubits//2-1)*I(nqubits-1))
ham37 = hamiltonians.SymbolicHamiltonian(form=Z(nqubits//2-1)*Z(nqubits-1))

In [11]:
def loss(params):
    cost = 0
    circuit = cnn(params)
    
    for j in range(num_data):
        final_state = circuit(gs_list[j]).state()

        z3 = np.real(ham3.expectation(final_state))
        z7 = np.real(ham7.expectation(final_state))
        zz37 = np.real(ham37.expectation(final_state))
        
        proj_00 = (1+zz37+z3+z7)/4
        proj_01 = (1-zz37-z3+z7)/4
        proj_10 = (1-zz37+z3-z7)/4
        proj_11 = (1+zz37-z3-z7)/4
        
        if label_list[j] == 0:
            cost += proj_00
        elif label_list[j] == 1:
            cost += proj_01
        elif label_list[j] == 2:
            cost += proj_10
        else:
            cost += proj_11

        # adding extra terms to equalize incorrect classes might help in the optimization, 
        # although it is not required
        """
        if label_list[j] == 0:
            cost += proj_00 + ((proj_01-proj_10)**2 + (proj_01-proj_11)**2 + (proj_10-proj_11)**2)/3
        elif label_list[j] == 1:
            cost += proj_01 + ((proj_00-proj_10)**2 + (proj_00-proj_11)**2 + (proj_10-proj_11)**2)/3
        elif label_list[j] == 2:
            cost += proj_10 + ((proj_00-proj_01)**2 + (proj_00-proj_11)**2 + (proj_01-proj_11)**2)/3
        else:
            cost += proj_11 + ((proj_00-proj_01)**2 + (proj_00-proj_10)**2 + (proj_01-proj_10)**2)/3
        """

    return cost/num_data

In [12]:
def accuracy(params):
    accuracy_data = 0
    circuit = cnn(params)
    
    for j in range(num_data):
        final_state = circuit(gs_list[j]).state()

        z3 = np.real(ham3.expectation(final_state))
        z7 = np.real(ham7.expectation(final_state))
        zz37 = np.real(ham37.expectation(final_state))
        
        proj_00 = (1+zz37+z3+z7)/4
        proj_01 = (1-zz37-z3+z7)/4
        proj_10 = (1-zz37+z3-z7)/4
        proj_11 = (1+zz37-z3-z7)/4
        
        if label_list[j] == 0:
            if proj_00 < proj_01 and proj_00 < proj_10 and proj_00 < proj_11:
                accuracy_data += 1
        elif label_list[j] == 1:
            if proj_01 < proj_00 and proj_01 < proj_10 and proj_01 < proj_11:
                accuracy_data += 1
        elif label_list[j] == 2:
            if proj_10 < proj_00 and proj_10 < proj_01 and proj_10 < proj_11:
                accuracy_data += 1
        else:
            if proj_11 < proj_00 and proj_11 < proj_01 and proj_11 < proj_10:
                accuracy_data += 1
            
    return accuracy_data*100/num_data

# Training

In [13]:
acc = 0
while acc < accuracy_training:
    initial_params = np.random.uniform(0, 2*np.pi, nparams)
    xopt = cma.fmin2(loss, initial_params, 0.7, options={'tolfun': 0.5e-3})
    print(xopt[1].result.fbest)
    print(xopt[1].result.xbest)
    
    np.savetxt(f"Results\\BEST_PARAMS_j1j2_{num_data}", [xopt[1].result.xbest], newline='')

    acc = accuracy(xopt[1].result.xbest)
    print(acc)

(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 15 (seed=203625, Wed Dec 20 09:57:53 2023)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]


    1     12 1.371307752117835e-01 1.0e+00 6.64e-01  7e-01  7e-01 0:00.0
    2     24 1.325080089439883e-01 1.1e+00 6.41e-01  6e-01  7e-01 0:00.1
    3     36 9.862341438811351e-02 1.2e+00 6.30e-01  6e-01  7e-01 0:00.1
   89   1068 8.204549655494454e-02 5.0e+00 4.38e-02  2e-02  5e-02 0:03.1
   91   1092 8.205307340593668e-02 5.3e+00 4.38e-02  2e-02  5e-02 0:03.2
termination on tolfun=0.0005 (Wed Dec 20 09:57:57 2023)
final/bestever f-value = 8.204860e-02 8.204550e-02 after 1093/1057 evaluations
incumbent solution: [0.06442296 3.40389136 5.87309442 3.10131909 5.67290122 2.63476213
 6.39136739 0.88840376 ...]
std deviations: [0.0290045  0.04013563 0.04916279 0.02397406 0.04696525 0.0462783
 0.05203237 0.02705282 ...]
0.08204549655494454
[0.02542942 3.39793682 5.87285203 3.12499831 5.71016628 2.68789808
 6.3881832  0.87279071 0.6485299  6.52692055 1.84451117 3.03717124
 3.38546474 6.25980624 1.31051792]
73.33333333333333
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 15 (seed=83577, Wed

KeyboardInterrupt: 